## Auditing and Exporting Detections

# REVISE ME
#### Before running this script, make sure that your Google Drive folder contains the orthomosaic GeoTiff (`step 0`) and no other GeoTIFF files, the `spatial_data.json` file (`step 1`), the `classes.csv` file (`step3`) and the `new_detections.json` file (`step 4`). If you want to add your new CNN detections to the manually annotated detections you created in VIA, also add the JSON file you exported using VIA (`step 2`). You will need to input that file name directly, since it is not standardized in our workflow.

<a href="https://colab.research.google.com/github/gl7176/GreySealCNN/blob/master/5_export_detections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<center> Be sure to update this hyperlink above if you clone and want to point to a different GitHub </center>

### Connect to our Google Drive folder and pull files
Note: when you run this it will give you a link that you must click. You must give Google some permissions, then copy a code into a box that comes up in the output section of this code.

If customizing this code, you will need to point the `drive_folder` variable to a URL for your shared google drive folder.

In [29]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1INuRNVKvKMy8L_Nb6lmoVbyvScWK0-0D'

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls key files from the directory specified above
#    and checks that all necessary files are present

orthomosaic_file = {}
spatial_data_file = local_download_path + '/spatial_data.json'
classes_file = local_download_path + '/classes.csv'
new_detections_file = local_download_path + '/new_detections.json'
subset_list_file = local_download_path + '/subset_list.csv'
via_annotations_file = local_download_path + '/via_SealCNN_TrainingData.csv'

checklist = ['orthomosaic', spatial_data_file, classes_file, new_detections_file, subset_list_file, via_annotations_file]

for f in file_list:
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  if fname.endswith(".tif") or fname.endswith(".json") or fname.endswith(".csv"):
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
    os.stat(fname)
    # if the file is a *.tif and larger than 100 mb we label it the orthomosaic
    if fname.endswith(".tif") and os.stat(fname).st_size > 10^8 :
      # if there are multiple orthomosaic files detected we spit an error
      if len(orthomosaic_file) != 0:
        raise Exception("more than one orthomosaic file identified based on size and type")
      orthomosaic_file = fname
      print("orthomosaic identified as " + orthomosaic_file)
      checklist.remove('orthomosaic')
    elif fname in checklist:
      checklist.remove(fname)
      print("required file found: " + fname)
    else:
      print("additional file found: " + fname)
print("all files pulled")
if len(checklist) > 0:
  print("Error: did not find " + str(len(checklist)) + " required files in your google folder: ", checklist)
  raise Exception("missing required data files")

required file found: data/spatial_data.json
required file found: data/subset_list.csv
required file found: data/new_detections.json
additional file found: data/annotations_valid.csv
additional file found: data/annotations_train.csv
additional file found: data/annotations_test.csv
required file found: data/classes.csv
required file found: data/via_SealCNN_TrainingData.csv
additional file found: data/via_SealCNN_TrainingData.json
orthomosaic identified as data/Hay Island 2015.tif
all files pulled


### Set up the python environment and key variables

In [30]:
import os
import argparse
import numpy as np
import json
import csv
!pip install rasterio
import rasterio
import copy

from shapely.geometry import mapping, Polygon
!pip install fiona
import fiona # only required for exporting to shapefiles

### Convert detections from image-/tile-based coordinates to orthomosaic coordinates

In [33]:
# open the output from our CNN
with open(new_detections_file) as f:
    detected_labels = json.load(f)

# open the output from our original tile splitting
with open(spatial_data_file) as f:
    spatial_data = json.load(f)
    tile_height = spatial_data["tile_height"]
    tile_width = spatial_data["tile_width"]
    img_data = spatial_data["tile_pointers"]
    
# update box locations from local tile coordinates to orthomosaic coordinates
# using image_locations from original tiling process, invoked by filename
# ---this involves transforming x1, y1, x2, y2 coordinates to 4 sets of x,y points
# and then transforming them back to the original form---
# then build a list of detection dictionaries. Note that filename is no longer
# needed because coordinates are now relative to the orthomosaic, not tile

detection_list = []
for key, value in detected_labels.items():
    for detection in value:
      # convert bounding box from x1/y1/x2/y2 format to [x1,y1],[x2,y1],[x2,y2],[x2,y1] coordinates format 
      bounding_box = np.array([[detection['box'][0], detection['box'][1]], [detection['box'][2], detection['box'][1]], [detection['box'][2], detection['box'][3]], [detection['box'][0], detection['box'][3]]])
      # update the new coordinates format from local tile coordinates to orthomosaic coordinates
      bounding_box = bounding_box + [img_data["image_locations"][[key][0].split("/")[-1]]]
      # convert our bounding box from coordinates format back to x1/y1/x2/y2 format
      bounding_box = [bounding_box[0][0], bounding_box[0][1], bounding_box[1][0], bounding_box[2][1]] 
      # update the dictionary
      detection['box'] = bounding_box
      #build our detection list. Note we no longer need filenames because the coordinates are no longer local
      detection_list.append(detection)
print(detection_list[0:3])

[{'box': [8419, 10931, 8473, 11012], 'label': 0, 'score': 0.8839489817619324}, {'box': [8488, 10814, 8531, 10852], 'label': 1, 'score': 0.8155218958854675}, {'box': [8864, 10193, 8905, 10227], 'label': 1, 'score': 0.6655842065811157}]


### Implement non-max suppression on duplicate CNN detections

In [34]:
# pull labels from classes.csv
import csv
with open(classes_file, "r") as f:
    reader = csv.reader(f, delimiter=",")
    labels_to_names = {int(i[1]):i[0] for i in reader}

boxes = []
scores = []
for detection in detection_list:
  boxes.append(detection['box'])
  scores.append(detection['score'])

In [35]:
# Malisiewicz et al.
# https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
# import the necessary packages
import numpy as np

def non_max_suppression(boxes, probs, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    # if probabilities are provided, sort on them instead
    if probs is not None:
        idxs = probs

    # sort the indexes
    idxs = np.argsort(idxs)

    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1, yy1, xx2, yy2 = np.maximum(x1[i], x1[idxs[:last]]), np.maximum(y1[i], y1[idxs[:last]]), np.minimum(x2[i], x2[idxs[:last]]), np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w, h = np.maximum(0, xx2 - xx1 + 1), np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return the index of the bounding boxes that were picked
    return pick

In [36]:
bboxes = np.array(boxes)
pick = non_max_suppression(bboxes, scores, 0.6)
nms_detection_list = []
for i in pick:
  nms_detection_list.append(detection_list[i])

# backup the nms_detection_list before running next section, so if something
# goes wrong we don't need to re-run the whole code
nms_detection_list_backup = copy.deepcopy(nms_detection_list)

print("Before NMS: " + str(len(detection_list)) + " detections")
print("After NMS: " + str(len(nms_detection_list)) + " detections")

Before NMS: 168 detections
After NMS: 158 detections


### Export CNN detections shapefile

In [37]:
# open the source image
dataset = rasterio.open(orthomosaic_file)

# the following section permanently alters nms_detection_list, necessitating the deepcopy backup earlier
nms_detection_list = copy.deepcopy(nms_detection_list_backup)

# converts orthomosaic pixel coordinates to global coordinates
geolocated_bb = []
for detection in nms_detection_list:
  bounding_box = np.array([[detection['box'][0], detection['box'][1]], [detection['box'][2], detection['box'][1]], [detection['box'][2], detection['box'][3]], [detection['box'][0], detection['box'][3]]]).astype(float)
  count = 0
  for point in bounding_box:
    point = dataset.transform * point
    bounding_box[count] = point
    count += 1
  detection['box'] = bounding_box
print(nms_detection_list[:3])

[{'box': array([[ 292120.37252, 5099996.81849],
       [ 292122.88762, 5099996.81849],
       [ 292122.88762, 5099994.33932],
       [ 292120.37252, 5099994.33932]]), 'label': 0, 'score': 0.9517046213150024}, {'box': array([[ 292167.47675, 5100079.709  ],
       [ 292170.31522, 5100079.709  ],
       [ 292170.31522, 5100077.94843],
       [ 292167.47675, 5100077.94843]]), 'label': 0, 'score': 0.9449800848960876}, {'box': array([[ 291971.29895, 5099918.16772],
       [ 291972.70022, 5099918.16772],
       [ 291972.70022, 5099916.9461 ],
       [ 291971.29895, 5099916.9461 ]]), 'label': 1, 'score': 0.9419155120849609}]


In [38]:
# write out the detections as a shapefile

from collections import OrderedDict
import fiona
from fiona.crs import from_epsg

# Set output directory
output_dir = 'shapefile_output'

# create the dir if it doesn't already exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str'),
        ('Score', 'float')
  ])
}

with fiona.open(output_dir + '/seal_detections.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(nms_detection_list):
      record = {
            'geometry': {'coordinates': [polygon['box']], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', orthomosaic_file),
                                       ('Detection', labels_to_names[polygon['label']]),
                                       ('Score', polygon['score'])
                                       ]),
            'type': 'Feature'}
      c.write(record)

### Zip output folder for download

In [39]:
# zip up the output directory into an archive for download
import subprocess
subprocess.call(['zip', '-r', '/content/' + output_dir + '.zip', '/content/' + output_dir])

from google.colab import files
files.download("/content/" + output_dir + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### RetinaNet to Existing VIA

In [ ]:
# open the previous training JSON data
with open(local_download_path + '/via_SealCNN_TrainingData.json') as f:
    existing_labels = json.load(f)

# add the new detections to the training JSON file
with open(new_detections_file) as f:
    detected_labels = json.load(f)
    
#"11_fiX1mEhK","[""2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---27.png""]",0,"[]","[2,509.275,929.174,89.376,48.904]","{}"

for filepath, detections in detected_labels.items():
    fn = filepath.split("/")[-1]
    # TODO is deep copy correct?
    annotations = existing_labels["_via_img_metadata"]
    for filename_size, metadata in annotations.items():
        print(metadata.items())
        if fn == metadata["filename"]:
            for detection in detections:
                # 'box' : [x1, y1, x2, y2]
                x1 = detection["box"][0]
                y1 = detection["box"][1]
                x2 = detection["box"][2]
                y2 = detection["box"][3]
                metadata["regions"].append({'shape_attributes': {'name': 'rect', 'x': x1, 'y': y1, 'width': x2-x1, 'height': y2-y1}, 'region_attributes': {}})

In [42]:
# write out new VIA file with additional detections

with open(local_download_path + '/via_region_data_detections.json', 'w') as fp:
    json.dump(existing_labels, fp)
    
from google.colab import files
files.download(local_download_path + "/via_region_data_detections.json")

# might be worth adding a section or conversion to allow this joined file to be converted to shapefile too.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### New Stuff: exporting VIA annotations as a shape file

In [ ]:
# make a dictionary where {filename} calls {subset type}
with open(subset_list_file, "r") as f:
    reader = csv.reader(f, delimiter=",")
    subset_dict = {i[0]:i[1] for i in reader}
    
via_annotations_list = []

# read each line, parse it, convert it, put it all back together
# then drop it in the appropriate subset
with open(via_annotations_file, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader: 
        # output we want:
        # format: path/to/image.jpg,x1,y1,x2,y2,class_name
        # example: /data/imgs/img_001.jpg,837,346,981,456,cow
        if 'filename' in line[0]:
            # bypassing comments in csv
            continue
        if '{}' in line[5]:
            #bypassing empty images
            continue
            
        filename = line[0]
        
        # pulling from column named "region_shape_attributes"
        box_entry = json.loads(line[5])
        top_left_x, top_left_y, width, height = box_entry["x"], box_entry["y"], box_entry["width"], box_entry["height"]
        if width == 0 or height == 0:
            continue
            # skip tiny/empty boxes
        
        # define area for NMS ranking later
        area = width * height

        # convert from "top left and width/height" to "x and y values at each corner of the box"
        if top_left_x < 0:
            top_left_x = 1
        if top_left_y < 0:
            top_left_y = 1
        x1 = top_left_x
        x2 = top_left_x + width
        y1 = top_left_y
        y2 = top_left_y + height 
        
        # pulling from column named "region_attributes" to get class names
        name = json.loads(line[6])["Age Class"]

        # skip unknown class, in this case. Might be useful in other applications though, e.g. total count
        if name == "Unknown":
            continue
        
        # pull subset from dictionary
        subset_type = subset_dict[filename]

        # create the annotation row
        new_row = []
        new_row.append(filename)
        new_row.append([x1,y1])
        new_row.append([x2,y1])
        new_row.append([x2,y2])
        new_row.append([x1,y2])
        new_row.append(name)
        new_row.append(area)
        new_row.append(subset_type)

        # append the row to the our list
        via_annotations_list.append(new_row)
print(via_annotations_list[0:5])

In [ ]:
# give each detection a score for NMS, based on its area and prioritized by
# testing > training datasets so during NMS if we encounter duplicates we prioritize
# testing data for accuracy evaluation, and within that prioritization, we prioritize
# larger boxes over smaller boxes (in case any "edge" animals were duplicated)
area_max = max(list(i[6] for i in via_annotations_list))
scores = []
boxes = []

detection_list = []
for i in via_annotations_list:
  score = i[6]/area_max
  if i[7] != 'testing':
    score = 0.01 * score
  scores.append(score)

  bounding_box = np.array([i[1],i[2],i[3],i[4]])
  # update the new coordinates format from local tile coordinates to orthomosaic coordinates
  bounding_box = bounding_box + [img_data["image_locations"][i[0]]]
  # convert our bounding box from coordinates format back to x1/y1/x2/y2 format
  bounding_box = [bounding_box[0][0], bounding_box[0][1], bounding_box[1][0], bounding_box[2][1]] 
  boxes.append(bounding_box)

  # update the dictionary
  detection = {"box":bounding_box, "score":score, "label":i[5], "subset":i[7]}
  #build our detection list. Note we no longer need filenames because the coordinates are no longer local
  detection_list.append(detection)
print(detection_list[0:3])

[{'box': [9815, 1847, 9881, 1879], 'score': 0.003300515705578997, 'label': 'Adult', 'subset': 'training'}, {'box': [10159, 1837, 10198, 1863], 'score': 0.001584622597280825, 'label': 'Pup', 'subset': 'training'}, {'box': [9832, 1819, 9868, 1852], 'score': 0.0018565400843881857, 'label': 'Pup', 'subset': 'training'}]


In [ ]:
bboxes = np.array(boxes)
pick = non_max_suppression(bboxes, scores, 0.6)
nms_detection_list = []
for i in pick:
  nms_detection_list.append(detection_list[i])

# backup the nms_detection_list before running next section, so if something
# goes wrong we don't need to re-run the whole code
nms_detection_list_backup = copy.deepcopy(nms_detection_list)

print("Before NMS: " + str(len(detection_list)) + " detections")
print("After NMS: " + str(len(nms_detection_list)) + " detections")

Before NMS: 2719 detections
After NMS: 2368 detections


### Export shapefile

In [ ]:
# open the source image
dataset = rasterio.open(orthomosaic_file)

# this section permanently alters nms_detection_list, necessitating the deepcopy backup earlier
nms_detection_list = copy.deepcopy(nms_detection_list_backup)
geolocated_bb = []
for detection in nms_detection_list:
  bounding_box = np.array([[detection['box'][0], detection['box'][1]], [detection['box'][2], detection['box'][1]], [detection['box'][2], detection['box'][3]], [detection['box'][0], detection['box'][3]]]).astype(float)
  count = 0
  for point in bounding_box:
    point = dataset.transform * point
    bounding_box[count] = point
    count += 1
  detection['box'] = bounding_box
print(nms_detection_list[0:3])

[{'box': array([[ 292015.24134, 5100059.30076],
       [ 292018.22353, 5100059.30076],
       [ 292018.22353, 5100056.85752],
       [ 292015.24134, 5100056.85752]]), 'score': 0.882012814502266, 'label': 'Adult', 'subset': 'testing'}, {'box': array([[ 291978.3053 , 5099904.22688],
       [ 291981.32342, 5099904.22688],
       [ 291981.32342, 5099901.96329],
       [ 291978.3053 , 5099901.96329]]), 'score': 0.8270042194092827, 'label': 'Adult', 'subset': 'testing'}, {'box': array([[ 292130.36106, 5100003.35775],
       [ 292132.51686, 5100003.35775],
       [ 292132.51686, 5100000.44742],
       [ 292130.36106, 5100000.44742]]), 'score': 0.759493670886076, 'label': 'Adult', 'subset': 'testing'}]


In [ ]:
# write out the detections as a shapefile

from collections import OrderedDict
import fiona
from fiona.crs import from_epsg

# Set output directory
output_dir = 'shapefile_output'

# create the dir if it doesn't already exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str'),
        ('Subset', 'str'),
  ])
}

with fiona.open(output_dir + '/seal_detections.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(nms_detection_list):
      record = {
            'geometry': {'coordinates': [polygon['box']], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', orthomosaic_file),
                                       ('Detection', polygon['label']),
                                       ('Subset', polygon['subset'])
                                       ]),
            'type': 'Feature'}
      c.write(record)

### Zip output folder for download

In [ ]:
# zip up the output directory into an archive for download
import subprocess
subprocess.call(['zip', '-r', '/content/' + output_dir + '.zip', '/content/' + output_dir])

from google.colab import files
files.download("/content/" + output_dir + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>